# Instagram posts generating AI-model

### **Task description**

Create an agent that accepts an instruction like "create a new post about the 'Adventure' model backpack with a 25-liter capacity for $200, which is great for climbers" or "write a post about a giveaway of 3 bags from the new collection" and generates an Instagram post in the style of this page: https://www.instagram.com/ospreypacks/.

### 0 - Import and install all necessary modules

In [ ]:
!pip install -q -U pandas torch transformers accelerate langchain langchain-huggingface gradio accelerate

In [ ]:
!pip uninstall torch torchvision -y
!pip install torch torchvision

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import gradio as gr
base_model = 'mistralai/Mistral-7B-Instruct-v0.2'

### 1 - Data Exploration

In [ ]:
#Load the dataset
instagram_data = '/kaggle/input/instagram-data/instagram_data.csv'
df = pd.read_csv(instagram_data, low_memory=False)
print(df.head())

In [ ]:
#Display basic information about the dataset
print(df.info())

#Check for missing values
print(df.isnull().sum())

#Display unique values in categorical columns
print(df['caption'].nunique())

In [ ]:
df_caption = df[['caption']]
df_caption.head()

In [ ]:
#Explore random captions
df_caption['caption'][1]

In [ ]:
df_caption['caption'][550]

In [ ]:
df_caption['caption'][2222]

In [ ]:
df_caption['caption'][201]

In [ ]:
df_caption['caption'][333]

### 2 - Initialize transformer from pre-trained model

In [ ]:
from huggingface_hub import login
HF_TOKEN = " "
login(token="HF_TOKEN")

In [ ]:
#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
my_pipeline = pipeline('text-generation',
                       model=base_model,
                       tokenizer=tokenizer,
                       torch_dtype=torch.bfloat16,
                       device_map='auto',
                       use_cache=True,
                       max_new_tokens=150
                       )

In [ ]:
my_llm = HuggingFacePipeline(pipeline=my_pipeline)

### 3 - Set description of the model's behaviour & Few-Shot Inference

In [ ]:
context = '''
You are an AI tasked with creating engaging Instagram post captions for Osprey Packs.
Your audience loves comfortable travel and outdoor adventures.
Highlight the product features creatively to appeal to this audience.
Use many emojis and hashtags in your caption.
Below are examples of input queries and their corresponding answers you have generated:
'''

In [ ]:
#Examples of queries and answers

examples = [
    {
        'query': "Generate a caption promoting the #RunHikeBike2022 giveaway. Mention how participants can enter by using the hashtag, following specific accounts for a bonus entry, and include the deadline for the giveaway. Ensure to highlight the excitement of the outdoor community coming together, and direct followers to the link in the bio for more details.",
        'answer': '''
#RunHikeBike2022 GIVEAWAY
Any day, anyway, any distance, solo, or in a group. The outdoor community is unifying across the country, joining together in person and on social media using #RunHikeBike2022
Show us how you share the trail and come together outside.
Hashtag #RunHikeBike2022 between March 15 and April 15 and be automatically entered to win this awesome prize package from the #RunHikeBike2022 team.

Bonus entry if you follow:
@ridecannondale
@vasquefootwear
@marmot
@biolite
@smartwool
@ospreypacks
@oiselle
@insolidarityproject

Giveaway closes April 15. Winner contacted and announced prior to April 22.
Tagging #RunHikeBIke2022 automatically enters you into the brand giveaway, eligible to participants over 18 years of age in the United States.

Learn more about #RunHikeBike2022 at the link in our bio!

#OspreyPacks
'''
    },
    {
        'query': "Create a caption that engages followers by asking them to comment on their favorite underrated National Park for summer travel",
        'answer': '''
Looking for summer travel inspo? Look no further.

Comment below your favorite underrated National Park to beat the crowds this summer!

#OspreyPacks
'''
    },
    {
        'query': "Generate an invitation post about Mountainfilm 2022 festival directing to the website link.",
        'answer': '''
It can be hard to put your finger on what exactly gives
you that Mountainfilm feeling. But something about these old festival intros comes very close.

Passes to Mountainfilm 2022 are on sale now! Whether you are able to join us
in-person or virtually, both festivals certainly promise to deliver that indescribable soul fire.
Get more info at mountainfilm.org - linked in our bio.

📷 @ben_eng_photo

#OspreyPacks #mountainfilm
#mountainfilm2022 #mountainfilmintelluride #mountainfilmonline ///
'''
    },
    {
        'query': 'Create a caption about an interview with a skier named Amber Chang.',
        'answer': '''
Stories to inspire your new year 🌞

What does it take to achieve 50 consecutive months of skiing? Skier Amber Chang (@amberkchang) tells us how she chases “turns all year”—from her home in the PNW to the peaks of Chile.

Read the stories that inspire us from #OspreyAmbassadors and #OspreyAthletes via the link in our bio. | #OspreyPacks ///
'''
    },
    {
        'query': "Write a caption about the pack from 'Jet' Series with some quotation.",
        'answer': '''
“Unbridled joy of accomplishment”. 📷 by: @digby_coffee
Featured pack from the Jet Series #ospreypacks #thegooddaysaremade ///
'''
    },
    {
        'query': "Write a caption promoting the inspiring stories from a Skier Amber Chang.",
        'answer': '''
Stories to inspire your new year 🌞

What does it take to achieve 50 consecutive months of skiing? Skier Amber Chang (@amberkchang)
tells us how she chases “turns all year”—from her home in the PNW to the peaks of Chile.

Read the stories that inspire us from #OspreyAmbassadors and #OspreyAthletes via the link in our bio. | #OspreyPacks ///
'''
    },

]

In [ ]:
#Define a prompt template that will be used for each example in the few-shot learning task
example_prompt = PromptTemplate(input_variables=['query', 'answer'],
                                template='''
Query: {query}
Answer: {answer}
'''
)

In [ ]:
#Define a prompt template that will be used for each example in the few-shot learning task
my_few_shot_prompt = FewShotPromptTemplate(examples=examples,
                                           example_prompt=example_prompt,
                                           prefix=context,
                                           suffix='Query: {query}\nAnswer:',
                                           input_variables=['query'],
                                           example_separator='\n')

In [ ]:
#Combine the few-shot prompt template with language model
my_llm_langchain = my_few_shot_prompt | my_llm

Now implement function for generating model's response.

In [ ]:
def gen_response(prompt: str, end_token='///'):
#     response = my_llm_langchain.invoke(prompt)
#   #Split the response by 'Answer: ' and keep the last part
#     answer_part = response.split('Answer: ')[-1]

#   # Split again by the end_token and take the first part
#     final_response = answer_part.split(end_token)[0].strip()

#     return final_response
    response = my_llm_chain.invoke(question)
    answer_start = response.rfind('Answer: ') + len('Answer: ')
    end_pos = response.find(end_token, answer_start)
    if end_pos != -1:
        final_response = response[answer_start:end_pos].strip()
    else:
        final_response = response[answer_start:].strip()
    return final_response


### 4 - Testing

In [ ]:
question_1 = '''
Generate a caption about our new backpack model called "Adventure" which has a capacity of 25 liters and costs only 200 $,
being a perfect match for passionate travellers.
'''
print(gen_response(question_1))

In [ ]:
question_2 = '''
Create a caption about hiking with some quote.
'''
print(gen_response(question_2))

In [ ]:
question_3 = '''
Generate a catchy caption about a new backpack model for kids called My First Mountain, in pink or green colours, using pink and green emojis there.
'''
print(gen_response(question_3))


### 5 - Implement simple UI

In [ ]:
ui = gr.Interface(fn=generate_response,
                  inputs=gr.components.Textbox(lines=5, placeholder="Enter your prompt here..."),
                  outputs='text',
                  title='Instagram Post Generator',
                  description='Please Enter a prompt to generate a custom Instagram post.')
ui.launch()